In [1]:
import pandas as pd #(etapa de extração)
import pandera as pa #a função do pandera é validar aos dados (faz parte da etapa de validação de dados)

In [21]:

df = pd.read_csv(
                "ocorrencia_2010_2020.csv", # in the df variable, load the csv file
                sep=";", 
                parse_dates=["ocorrencia_dia"], #change the type, from object to date
                dayfirst=True # in american format, day is set as first element
)


df.head(10)    #print the first 10 rows of the dataframe
# df.tail(10)  #print the first 10 rows of the dataframe

#Attention: If we have a wrong date, Pandas cannot apply the "parse_dates" command to correct the date.  Wrong date example:  50/50/2021

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,3:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,****,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-01-10,20:00:00,0


In [3]:
#no schema você define manualmente o tipo de dados de cada coluna deveria ter, para comparar com a original. Com isso, conseguimos visualizar erros de dados em tempo de execução para corrigi-los 

schema = pa.DataFrameSchema(
    columns = {
        "codigo": pa.Column(pa.Int, required=False), #por padrão as colunas são setadas como True, mas podemos definir se ela é obrigatória ou não. No caso essa coluna não existe na nossa base
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)), #tamanha máximo e minimo de caracteres
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }

)

# ^    => o ^ indica que a expressão(palavra ou numero etc) precisa inicir com essa regra "([0-1]?[0-9]|[2][0-3])"
# $    => o $ indica que a expressão(palavra ou numero etc) precisa terminar com a regra "(:[0-5][0-9])?"

# ?    => indica que a regra anterior ao ? pode acontecer 1 vez ou nenhuma (permite nulo). 
            #Ex1. [0-1]? o primeiro digito pode ser 0, 1 ou nulo. 
            #Ex2 (:[0-5][0-9])? a expressão deve iniciar com ":" seguida de um numero entre 0 e 5 e depois de outro entre 0 e 9. Tudo isso pode acontecer ou não! (pode ter o segundo ou não)   

# |        => uma ou outra ocorrência
#[2][0-3]  => primeiro digito é 2 e o segundo pode ser entre 0 e 3


In [4]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,3:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5735,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5736,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5737,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5738,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,0:54:00,0


In [5]:
df.dtypes
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5735    12
5736    12
5737    12
5738    12
5739    12
Name: ocorrencia_dia, Length: 5740, dtype: int64